In [35]:
import numpy as np

In [36]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge

In [37]:
class Stacking:
    def __init__(self, model=DecisionTreeRegressor, depth=10, meta_model=Ridge):
        self.meta_model = meta_model
        self.model_type = model
        self.models = {}
        self.depth = depth
        self.meta = None
    
    def fit(self, X, y):
        predictions = []
        for iter in range(self.depth):
            self.models[iter] = self.model_type(max_depth=5)
            self.models[iter].fit(X, y)
            predictions.append(self.models[iter].predict(X))
        self.meta = self.meta_model(alpha=1.0)
        self.meta.fit(np.array(predictions).T, y)
    
    def predict(self, X):
        predictions = []
        for iter in self.models.keys():
            predictions.append(self.models[iter].predict(X))
        return self.meta.predict(np.array(predictions).T)

In [38]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

# Создаем синтетические данные
X, y = make_regression(n_samples=1000, n_features=20, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Создаем и обучаем модель стекинга
stacking_model = Stacking(model=DecisionTreeRegressor, depth=5)
stacking_model.fit(X_train, y_train)

# Делаем предсказания на тестовых данных
y_pred = stacking_model.predict(X_test)

# Оцениваем качество модели с помощью среднеквадратичной ошибки (MSE)
mse = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mse)

Mean Absolute Error (MAE): 110.83398369462924
